
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>




# Decision Trees

In the previous notebook, you were working with the parametric model, Linear Regression. We could do some more hyperparameter tuning with the linear regression model, but we're going to try tree based methods and see if our performance improves.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Learning Objectives:<br>


By the end of this lesson, you should be able to;

* Build a decision tree model using Spark ML
* Identify the differences between single-node and distributed decision tree implementations
* View and interpret feature importance in decision tree models
* Discuss common pitfalls of decision tree models

## 📌 Requirements

**Required Databricks Runtime Version:** 
* Please note that in order to run this notebook, you must use one of the following Databricks Runtime(s): **12.2.x-cpu-ml-scala2.12**

## Lesson Setup

The first thing we're going to do is to **run setup script**. This script will define the required configuration variables that are scoped to each user.

In [0]:
%run "./Includes/Classroom-Setup"

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02"

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)

Creating & using the schema "charlie_ohara_4mi2_da_sml" in the catalog "hive_metastore"...(1 seconds)

Predefined tables in "charlie_ohara_4mi2_da_sml":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02

Setup completed (12 seconds)


In [0]:
file_path = "dbfs:/mnt/dbacademy-datasets/scalable-machine-learning-with-apache-spark/v02/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"
airbnb_df = spark.read.format("delta").load(file_path)
train_df, test_df = airbnb_df.randomSplit([.8, .2], seed=42)




## How to Handle Categorical Features?

We saw in the previous notebook that we can use StringIndexer/OneHotEncoder/VectorAssembler or RFormula.

**However, for decision trees, and in particular, random forests, we should not OHE our variables.**

There is an excellent <a href="https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769#:~:text=One%2Dhot%20encoding%20categorical%20variables,importance%20resulting%20in%20poorer%20performance" target="_blank">blog</a> on this, and the essence is:
>>> "One-hot encoding categorical variables with high cardinality can cause inefficiency in tree-based methods. Continuous variables will be given more importance than the dummy variables by the algorithm, which will obscure the order of feature importance and can result in poorer performance."

In [0]:
from pyspark.ml.feature import StringIndexer

categorical_cols = [field for (field, dataType) in train_df.dtypes if dataType == "string"]
index_output_cols = [x + "Index" for x in categorical_cols]
string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols, handleInvalid="skip")




## VectorAssembler

Let's use the <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler" target="_blank">VectorAssembler</a> to combine all of our categorical and numeric inputs.

In [0]:
from pyspark.ml.feature import VectorAssembler

# Filter for just numeric columns (and exclude price, our label)
numeric_cols = [field for (field, dataType) in train_df.dtypes if ((dataType == "double") & (field != "price"))]
# Combine output of StringIndexer defined above and numeric columns
assembler_inputs = index_output_cols + numeric_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")




## Decision Tree

Now let's build a <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.DecisionTreeRegressor.html?highlight=decisiontreeregressor#pyspark.ml.regression.DecisionTreeRegressor" target="_blank">DecisionTreeRegressor</a> with the default hyperparameters.

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

# this is where we define the algorithm we want to use 
dt = DecisionTreeRegressor(labelCol="price")




## Fit Pipeline

The following cell is expected to error, but we subsequently fix this.

In [0]:
from pyspark.ml import Pipeline

# Combine stages into pipeline
stages = [string_indexer, vec_assembler, dt]
pipeline = Pipeline(stages=stages)

# Each worker will only get a couple of values, not all the unique values
pipeline_model = pipeline.fit(train_df)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-3021276973846994>:8
      5 pipeline = Pipeline(stages=stages)
      7 # Uncomment to perform fit
----> 8 pipeline_model = pipeline.fit(train_df)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/spark/python/pyspark/ml/base.py:205, in Estimator.fit(self, dataset, params)
    203         return self.copy(params)._fit(dataset)
    204     else:
--> 205         return self._fit(dataset)
    206 else:
    207     raise TypeError(
    208         "Params must be either a param map or a list/tuple of param maps, "
    209         "but got %s." 

In [0]:
# maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
# can't fit 36 things into 32 bins
print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label, current: price)
leafCol: Leaf indices column name. Predicted leaf index of each instance in each tree by preorder. (default: )
maxBins: Max number of bins for discretizing continuous feat

In [0]:
# Given this error - requirement failed: DecisionTree requires maxBins (= 32) to be at least as large as the number of values in each categorical feature, but categorical feature 3 has 36 values. Consider removing this and other categorical features with a large number of values, or add more training examples.
# We want to identify feature 3 = index in a list
assembler_inputs[3] # neighbourhood_cleansedIndex

Out[17]: 'neighbourhood_cleansedIndex'

In [0]:
# we then count the number of unique values in the neighbourhood column and see there are 36
train_df.groupBy("neighbourhood_cleansed").count().count() 

Out[16]: 36




## maxBins

What is this parameter <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.DecisionTreeRegressor.html?highlight=decisiontreeregressor#pyspark.ml.regression.DecisionTreeRegressor.maxBins" target="_blank">maxBins</a>? Let's take a look at the PLANET implementation of distributed decision trees to help explain the **`maxBins`** parameter.

Spark separates the data into different workers, so each worker may only have  a couple of the categories. Makes it difficult to identify which categories matter if the worker can't see them all.





<img src="https://files.training.databricks.com/images/DistDecisionTrees.png" height=500px>




In Spark, data is partitioned by row. So when it needs to make a split, each worker has to compute summary statistics for every feature for  each split point. Then these summary statistics have to be aggregated (via tree reduce) for a split to be made. 

Think about it: What if worker 1 had the value **`32`** but none of the others had it. How could you communicate how good of a split that would be? So, Spark has a maxBins parameter for discretizing continuous variables into buckets, but the **number of buckets has to be as large as the categorical variable with the highest cardinality.**





Let's go ahead and increase maxBins to **`40`**.

In [0]:
dt.setMaxBins(40)

Out[19]: DecisionTreeRegressor_a491ee3bb13d




Take two.

In [0]:
pipeline_model = pipeline.fit(train_df)




## Feature Importance

Let's go ahead and get the fitted decision tree model, and look at the feature importance scores.

In [0]:
dt_model = pipeline_model.stages[-1]
display(dt_model) # See that feature 12 is the most valuable feature in the initial decision

treeNode
"{""index"":31,""featureType"":""continuous"",""prediction"":null,""threshold"":2.5,""categories"":null,""feature"":12,""overflow"":false}"
"{""index"":15,""featureType"":""continuous"",""prediction"":null,""threshold"":1.5,""categories"":null,""feature"":12,""overflow"":false}"
"{""index"":7,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[1.0,2.0],""feature"":5,""overflow"":false}"
"{""index"":3,""featureType"":""categorical"",""prediction"":null,""threshold"":null,""categories"":[5.0,6.0,7.0,8.0,9.0,10.0,12.0,13.0,14.0,17.0,18.0,19.0,23.0,24.0,26.0,27.0,28.0,29.0,30.0,31.0,33.0,35.0],""feature"":3,""overflow"":false}"
"{""index"":1,""featureType"":""continuous"",""prediction"":null,""threshold"":8.5,""categories"":null,""feature"":10,""overflow"":false}"
"{""index"":0,""featureType"":null,""prediction"":96.27024390243902,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":2,""featureType"":null,""prediction"":984.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":5,""featureType"":""continuous"",""prediction"":null,""threshold"":37.744265,""categories"":null,""feature"":8,""overflow"":false}"
"{""index"":4,""featureType"":null,""prediction"":343.0,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"
"{""index"":6,""featureType"":null,""prediction"":136.60329067641683,""threshold"":null,""categories"":null,""feature"":null,""overflow"":false}"


In [0]:
dt_model.featureImportances

Out[22]: SparseVector(33, {0: 0.0179, 1: 0.1551, 2: 0.1295, 3: 0.1013, 5: 0.0106, 8: 0.0035, 10: 0.0604, 12: 0.2111, 13: 0.0174, 14: 0.1433, 15: 0.1134, 22: 0.0365})




### Interpreting Feature Importance

Hmmm... it's a little hard to know what feature 4 vs 11 is. Given that the feature importance scores are "small data", let's use Pandas to help us recover the original column names.

In [0]:
import pandas as pd

features_df = pd.DataFrame(list(zip(vec_assembler.getInputCols(), dt_model.featureImportances)), columns=["feature", "importance"])
features_df

,feature,importance
0,host_is_superhostIndex,0.017857
1,cancellation_policyIndex,0.155079
2,instant_bookableIndex,0.129501
3,neighbourhood_cleansedIndex,0.101304
4,property_typeIndex,0.000000
5,room_typeIndex,0.010584
6,bed_typeIndex,0.000000
7,host_total_listings_count,0.000000
8,latitude,0.003535
9,longitude,0.000000





### Why so few features are non-zero?

With SparkML, the default **`maxDepth`** is 5, so there are only a few features we could consider (we can also split on the same feature many times at different split points).

Let's use a Databricks widget to get the top-K features.

In [0]:
dbutils.widgets.text("top_k", "5")
top_k = int(dbutils.widgets.get("top_k"))

top_features = features_df.sort_values(["importance"], ascending=False)[:top_k]["feature"].values
print(top_features) # most important features for determining price

['bedrooms' 'cancellation_policyIndex' 'minimum_nights'
 'instant_bookableIndex' 'number_of_reviews']





## Scale Invariant

With decision trees, the scale of the features does not matter. For example, it will split 1/3 of the data if that split point is 100 or if it is normalized to be .33. The only thing that matters is how many data points fall left and right of that split point - not the absolute value of the split point.

This is not true for linear regression, and the default in Spark is to standardize first. Think about it: If you measure shoe sizes in American vs European sizing, the corresponding weight of those features will be very different even those those measures represent the same thing: the size of a person's foot!




## Apply model to test set

In [0]:
pred_df = pipeline_model.transform(test_df)

# We see our model is susceptible to outliers 
display(pred_df.select("features", "price", "prediction").orderBy("price", ascending=False))

features,price,prediction
"Map(vectorType -> dense, length -> 33, values -> List(0.0, 0.0, 1.0, 21.0, 0.0, 0.0, 0.0, 165.0, 37.78778, -122.39426, 2.0, 1.0, 0.0, 1.0, 30.0, 0.0, 98.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1599.0,202.86818632309217
"Map(vectorType -> sparse, length -> 33, indices -> List(1, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(2.0, 9.0, 1.0, 1.0, 37.75508, -122.40173, 10.0, 3.5, 4.0, 5.0, 4.0, 3.0, 100.0, 10.0, 9.0, 10.0, 10.0, 9.0, 10.0))",1450.0,486.4035087719298
"Map(vectorType -> dense, length -> 33, values -> List(0.0, 0.0, 0.0, 10.0, 2.0, 0.0, 0.0, 1.0, 37.79271, -122.4108, 6.0, 2.5, 3.0, 4.0, 3.0, 1.0, 98.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1309.0,740.3870967741935
"Map(vectorType -> sparse, length -> 33, indices -> List(1, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(5.0, 21.0, 7.0, 16.0, 37.78889, -122.40358, 8.0, 3.5, 3.0, 4.0, 1.0, 2.0, 100.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0))",1252.0,8000.0
"Map(vectorType -> sparse, length -> 33, indices -> List(1, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(1.0, 16.0, 1.0, 1.0, 37.79067, -122.42977, 6.0, 2.5, 3.0, 4.0, 2.0, 13.0, 100.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0))",1250.0,740.3870967741935
"Map(vectorType -> dense, length -> 33, values -> List(1.0, 0.0, 1.0, 3.0, 7.0, 0.0, 0.0, 10.0, 37.78858, -122.41331, 4.0, 1.0, 1.0, 2.0, 4.0, 0.0, 98.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))",1200.0,202.86818632309217
"Map(vectorType -> sparse, length -> 33, indices -> List(1, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(1.0, 6.0, 2.0, 37.77245, -122.44198, 6.0, 1.5, 3.0, 3.0, 3.0, 3.0, 100.0, 10.0, 10.0, 10.0, 10.0, 10.0, 9.0))",1195.0,486.4035087719298
"Map(vectorType -> sparse, length -> 33, indices -> List(3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(2.0, 2.0, 37.77658, -122.43663, 8.0, 2.0, 4.0, 5.0, 30.0, 9.0, 97.0, 10.0, 10.0, 10.0, 10.0, 10.0, 8.0))",1150.0,300.0
"Map(vectorType -> sparse, length -> 33, indices -> List(3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(15.0, 2.0, 1.0, 37.80465, -122.42262, 6.0, 2.0, 2.0, 2.0, 1.0, 8.0, 97.0, 10.0, 9.0, 9.0, 10.0, 10.0, 9.0))",1099.0,408.10526315789474
"Map(vectorType -> sparse, length -> 33, indices -> List(4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), values -> List(1.0, 1.0, 37.75993, -122.42555, 10.0, 2.0, 6.0, 7.0, 2.0, 5.0, 100.0, 9.0, 10.0, 10.0, 10.0, 10.0, 9.0))",1080.0,486.4035087719298


Databricks visualization. Run in Databricks to view.




## Pitfall

What if we get a massive Airbnb rental? It was 20 bedrooms and 20 bathrooms. What will a decision tree predict?

It turns out decision trees cannot predict any values larger than they were trained on. The max value in our training set was $10,000, so we can't predict any values larger than that.

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

regression_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")

rmse = regression_evaluator.evaluate(pred_df)
r2 = regression_evaluator.setMetricName("r2").evaluate(pred_df)
print(f"RMSE is {rmse}")
print(f"R2 is {r2}")

RMSE is 409.3381256547409
R2 is -4.2526295883320735





## Uh oh!

This model is way worse than the linear regression model, and it's even worse than just predicting the average value.

In the next few notebooks, let's look at hyperparameter tuning and ensemble models to improve upon the performance of our single decision tree.


## Classroom Cleanup

Run the following cell to remove lessons-specific assets created during this lesson:

In [0]:
DA.cleanup()

Resetting the learning environment:
| dropping the schema "charlie_ohara_4mi2_da_sml"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/charlie.ohara@standard.ai/scalable-machine-learning-with-apache-spark"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(3 seconds)
| validation completed...(3 seconds total)


&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>